In [2]:
from initial_functions import fn_read_from_db, fn_write_to_db
import pandas as pd
from sqlalchemy import create_engine

In [3]:
engine_local = create_engine('sqlite:///trade_ai.db')

In [4]:
#read from db
def fn_read_from_db(table_name, columns=None, where=None):
    print(engine_local)
    # SELECT cümlesi oluştur
    cols = ", ".join(columns) if columns else "*"
    sql = f"SELECT {cols} FROM {table_name}"
    
    if where:
        sql += f" WHERE {where}"
    
    # pandas ile SQL sorgusu çalıştır
    df = pd.read_sql(sql, con=engine_local)
    return df

In [5]:
df_raw = fn_read_from_db('tbl_ticker_actual_daily_2')

Engine(sqlite:///trade_ai.db)


In [27]:
TICKER = 'ULKER.IS'

In [29]:
# df_ticker = df_raw[df_raw['TICKER']=='ULKER.IS']

In [31]:
def calculate_all_ranges(df, ticker, interval,CUTT_OFF=None):
    df = df.copy()
    df = df[df['TICKER']==ticker]
    df['DATETIME'] = pd.to_datetime(df['DATETIME'])

    if CUTT_OFF is not None:
        # 1. string → datetime (dayfirst=True)
        cut_dt = pd.to_datetime(CUTT_OFF, dayfirst=True)
    
        # 2. df['DATETIME'] timezone-aware ise CUTT_OFF'u aynı tz yap
        if df['DATETIME'].dt.tz is not None:
            cut_dt = cut_dt.tz_localize(df['DATETIME'].dt.tz)
    
        # 3. Filtrele
        df = df[df['DATETIME'] <= cut_dt]

    # En yeniden eskiye sırala
    df = df.sort_values('DATETIME', ascending=False).reset_index(drop=True)

    # Eğer filtre sonrası boşsa direkt boş dön
    if df.empty:
        return pd.DataFrame()

    end_datetime = df.iloc[0]['DATETIME']

    range_map = {
        "1day": pd.DateOffset(days=1),
        "1week": pd.DateOffset(weeks=1),
        "1months": pd.DateOffset(months=1),
        "3months": pd.DateOffset(months=3),
        "6months": pd.DateOffset(months=6),
        "1year": pd.DateOffset(years=1),
        "2year": pd.DateOffset(years=2)
    }

    results = []

    for range_name, offset in range_map.items():

        range_startdate = end_datetime - offset

        df_filter = df[
            (df['DATETIME'] >= range_startdate) &
            (df['DATETIME'] <= end_datetime)
        ].copy()

        if df_filter.empty:
            continue

        range_size = len(df_filter)

        highest_idx = df_filter['HIGH'].idxmax()
        highest_value = round(df_filter.loc[highest_idx, 'HIGH'], 2)
        highest_date = df_filter.loc[highest_idx, 'DATETIME']

        # Highest sonrası (artık CUTT_OFF kontrolü gerekmiyor)
        df_after = df_filter[df_filter['DATETIME'] >= highest_date]
        row_size_after_highest = len(df_after)

        end_datetime_filtered = df_filter['DATETIME'].max()

   

        results.append({
            "TICKER": ticker,
            "INTERVAL":interval,
            "RANGE_TYPE": range_name,
            "RANGE_STARTDATE": range_startdate,
            "RANGE_SIZE": range_size,
            "HIGHEST_VALUE": highest_value,
            "HIGHEST_DATE": highest_date,
            "ROW_SIZE_A_H": row_size_after_highest,
            "CUTT_OFF_DATE":CUTT_OFF,
            "END_DATETIME": end_datetime_filtered
        })

    df_res = pd.DataFrame(results)
    return df_res

In [33]:
df_res = calculate_all_ranges(df_raw, TICKER,'daily', CUTT_OFF=None)

In [34]:
import pandas as pd
import numpy as np

def calculate_tv_frvp_gpt(ticker, df_ohlc, df_dates, value_area_pct=68):
    """
    TradingView FRVP mantığına uygun hesaplama.

    ticker: İşlem yapmak istediğin TICKER
    df_ohlc: OHLCV verisi (DATETIME, TICKER, OPEN, HIGH, LOW, CLOSE, VOLUME)
    df_dates: FRVP başlangıç tarihleri (TICKER, HIGHEST_DATE)
    value_area_pct: Value Area yüzdesi (TradingView parametresi, default=68)

    Döndürür: df_dates ile FRVP POC, VAL, VAH eklenmiş hali
    """
    
    # df_dates'den sadece ilgili ticker'ı filtrele
    df_ticker_dates = df_dates[df_dates['TICKER'] == ticker].copy()
    if df_ticker_dates.empty:
        raise ValueError(f"{ticker} için başlangıç tarihi bulunamadı.")
    
    # df_ohlc'de sadece ilgili ticker'ı al
    df_ohlc_ticker = df_ohlc[df_ohlc['TICKER'] == ticker].copy()
    if df_ohlc_ticker.empty:
        raise ValueError(f"{ticker} için OHLC verisi bulunamadı.")
    
    # DATETIME sütununu datetime tipine çevir
    df_ohlc_ticker['DATETIME'] = pd.to_datetime(df_ohlc_ticker['DATETIME'])
    
    # Çıktı df
    df_result = df_ticker_dates.copy()
    df_result['FRVP_POC'] = np.nan
    df_result['FRVP_VAL'] = np.nan
    df_result['FRVP_VAH'] = np.nan
    
    # Her HIGHEST_DATE için FRVP hesapla
    for idx, row in df_ticker_dates.iterrows():
        start_date = pd.to_datetime(row['HIGHEST_DATE'])
        
        # HIGHEST_DATE'ten itibaren veri al
        df_period = df_ohlc_ticker[df_ohlc_ticker['DATETIME'] >= start_date].copy()
        if df_period.empty:
            continue
        
        # Fiyat aralığı
        min_price = df_period['LOW'].min()
        max_price = df_period['HIGH'].max()
        price_bins = np.arange(min_price, max_price + 1, 1)  # Row Size = 1 tick
        
        # Volume profile histogram
        hist = pd.Series(0, index=price_bins)
        for _, bar in df_period.iterrows():
            low = bar['LOW']
            high = bar['HIGH']
            vol = bar['VOLUME']
            overlap = price_bins[(price_bins >= low) & (price_bins <= high)]
            if len(overlap) > 0:
                hist.loc[overlap] += vol / len(overlap)
        
        # POC
        poc = hist.idxmax()
        
        # Value Area (VAL, VAH)
        total_volume = hist.sum()
        target_volume = total_volume * (value_area_pct / 100)
        
        hist_sorted = hist.sort_index()
        cum_vol = hist_sorted.copy()
        cum_vol[:] = 0
        
        cum_vol[poc] = hist_sorted[poc]
        lower_idx = hist_sorted.index.get_loc(poc) - 1
        upper_idx = hist_sorted.index.get_loc(poc) + 1
        cum_total = hist_sorted[poc]
        
        while cum_total < target_volume:
            lower_vol = hist_sorted.iloc[lower_idx] if lower_idx >= 0 else 0
            upper_vol = hist_sorted.iloc[upper_idx] if upper_idx < len(hist_sorted) else 0
            
            if lower_vol >= upper_vol:
                cum_total += lower_vol
                cum_vol.iloc[lower_idx] = lower_vol
                lower_idx -= 1
            else:
                cum_total += upper_vol
                cum_vol.iloc[upper_idx] = upper_vol
                upper_idx += 1
            
            if lower_idx < 0 and upper_idx >= len(hist_sorted):
                break
        
        val = cum_vol[cum_vol > 0].index.min()
        vah = cum_vol[cum_vol > 0].index.max()
        
        # Sonuçları df_result'e ekle
        df_result.loc[idx, 'FRVP_POC'] = round(poc,2)
        df_result.loc[idx, 'FRVP_VAL'] = round(val,2)
        df_result.loc[idx, 'FRVP_VAH'] = round(vah,2)
    
    return df_result

df_result = calculate_tv_frvp_gpt(TICKER,df_raw, df_res, value_area_pct=68)
df_result

,TICKER,INTERVAL,RANGE_TYPE,RANGE_STARTDATE,RANGE_SIZE,HIGHEST_VALUE,HIGHEST_DATE,ROW_SIZE_A_H,CUTT_OFF_DATE,END_DATETIME,FRVP_POC,FRVP_VAL,FRVP_VAH
0,ULKER.IS,daily,1day,2026-02-10 00:00:00+03:00,2,138.00,2026-02-10 00:00:00+03:00,2,None,2026-02-11 00:00:00+03:00,132.80,130.80,134.80
1,ULKER.IS,daily,1week,2026-02-04 00:00:00+03:00,6,139.50,2026-02-04 00:00:00+03:00,6,None,2026-02-11 00:00:00+03:00,133.80,132.80,136.80
2,ULKER.IS,daily,1months,2026-01-11 00:00:00+03:00,23,139.50,2026-02-04 00:00:00+03:00,6,None,2026-02-11 00:00:00+03:00,133.80,132.80,136.80
3,ULKER.IS,daily,3months,2025-11-11 00:00:00+03:00,66,139.50,2026-02-04 00:00:00+03:00,6,None,2026-02-11 00:00:00+03:00,133.80,132.80,136.80
4,ULKER.IS,daily,6months,2025-08-11 00:00:00+03:00,131,139.50,2026-02-04 00:00:00+03:00,6,None,2026-02-11 00:00:00+03:00,133.80,132.80,136.80
5,ULKER.IS,daily,1year,2025-02-11 00:00:00+03:00,252,139.50,2026-02-04 00:00:00+03:00,6,None,2026-02-11 00:00:00+03:00,133.80,132.80,136.80
6,ULKER.IS,daily,2year,2024-02-11 00:00:00+03:00,500,174.64,2024-07-16 00:00:00+03:00,399,None,2026-02-11 00:00:00+03:00,110.27,100.27,115.27


In [43]:
import pandas as pd
import numpy as np

def calculate_tv_frvp_gpt(ticker, df_ohlc, df_dates, value_area_pct=68):
    """
    TradingView FRVP mantığına %100 uyumlu hesaplama
    
    ticker: İşlem yapmak istediğin TICKER
    df_ohlc: OHLCV verisi (DATETIME, TICKER, OPEN, HIGH, LOW, CLOSE, VOLUME)
    df_dates: FRVP başlangıç tarihleri (TICKER, HIGHEST_DATE)
    value_area_pct: Value Area yüzdesi (TradingView parametresi, default=68)
    
    Döndürür: df_dates ile FRVP POC, VAL, VAH eklenmiş hali
    """
    
    # df_dates'den sadece ilgili ticker'ı filtrele
    df_ticker_dates = df_dates[df_dates['TICKER'] == ticker].copy()
    if df_ticker_dates.empty:
        raise ValueError(f"{ticker} için başlangıç tarihi bulunamadı.")
    
    # df_ohlc'de sadece ilgili ticker'ı al
    df_ohlc_ticker = df_ohlc[df_ohlc['TICKER'] == ticker].copy()
    if df_ohlc_ticker.empty:
        raise ValueError(f"{ticker} için OHLC verisi bulunamadı.")
    
    # DATETIME sütununu datetime tipine çevir
    df_ohlc_ticker['DATETIME'] = pd.to_datetime(df_ohlc_ticker['DATETIME'])
    
    # Timezone varsa kaldır
    if hasattr(df_ohlc_ticker['DATETIME'].dt, 'tz') and df_ohlc_ticker['DATETIME'].dt.tz is not None:
        df_ohlc_ticker['DATETIME'] = df_ohlc_ticker['DATETIME'].dt.tz_localize(None)
    
    df_ohlc_ticker = df_ohlc_ticker.sort_values('DATETIME').reset_index(drop=True)
    
    # TRADINGVIEW TICK SIZE - En önemli düzeltme!
    # TradingView hisse senedi fiyatlarına göre tick size belirler
    sample_price = df_ohlc_ticker['CLOSE'].iloc[0]
    if sample_price >= 1000:
        tick_size = 1.0
    elif sample_price >= 100:
        tick_size = 0.1
    elif sample_price >= 10:
        tick_size = 0.01
    elif sample_price >= 1:
        tick_size = 0.01  # 1-10 arası hisseler de 0.01 tick
    elif sample_price >= 0.1:
        tick_size = 0.001
    else:
        tick_size = 0.0001
    
    print(f"Ticker: {ticker}, Tick Size: {tick_size}, Sample Price: {sample_price}")
    
    # Çıktı df
    df_result = df_ticker_dates.copy()
    df_result['FRVP_POC'] = np.nan
    df_result['FRVP_VAL'] = np.nan
    df_result['FRVP_VAH'] = np.nan
    
    # Başlangıç tarihlerini datetime'a çevir
    df_ticker_dates['HIGHEST_DATE'] = pd.to_datetime(df_ticker_dates['HIGHEST_DATE'])
    
    # Timezone'ları temizle
    if hasattr(df_ticker_dates['HIGHEST_DATE'].dt, 'tz') and df_ticker_dates['HIGHEST_DATE'].dt.tz is not None:
        df_ticker_dates['HIGHEST_DATE'] = df_ticker_dates['HIGHEST_DATE'].dt.tz_localize(None)
    
    # Her HIGHEST_DATE için FRVP hesapla
    for idx, row in df_ticker_dates.iterrows():
        start_date = row['HIGHEST_DATE']
        
        # HIGHEST_DATE'ten itibaren veri al
        mask = df_ohlc_ticker['DATETIME'] >= start_date
        df_period = df_ohlc_ticker[mask].copy()
        
        if len(df_period) == 0:
            continue
        
        # Fiyat aralığını TradingView mantığıyla belirle
        min_price = df_period['LOW'].min()
        max_price = df_period['HIGH'].max()
        
        # Tick size'a göre yuvarla
        min_price = np.floor(min_price / tick_size) * tick_size
        max_price = np.ceil(max_price / tick_size) * tick_size
        
        # Fiyat seviyeleri - TradingView gibi tam tick_size katları
        n_bins = int(round((max_price - min_price) / tick_size)) + 1
        price_levels = np.round(np.linspace(min_price, max_price, n_bins), 6)
        
        # Volume profile histogram
        hist = np.zeros(len(price_levels))
        
        # TradingView'un hacim dağıtım algoritması
        for _, bar in df_period.iterrows():
            low = bar['LOW']
            high = bar['HIGH']
            vol = bar['VOLUME']
            
            # En yakın tick_size katlarına yuvarla
            low_tick = np.round(low / tick_size) * tick_size
            high_tick = np.round(high / tick_size) * tick_size
            
            # TradingView: Fiyat aralığındaki tick sayısı
            n_ticks = int(round((high_tick - low_tick) / tick_size)) + 1
            
            if n_ticks > 0:
                vol_per_tick = vol / n_ticks
                
                # Hangi fiyat seviyelerine denk geliyor?
                tick_prices = np.round(np.linspace(low_tick, high_tick, n_ticks), 6)
                
                for tick_price in tick_prices:
                    # En yakın price_levels indeksini bul
                    closest_idx = np.argmin(np.abs(price_levels - tick_price))
                    if abs(price_levels[closest_idx] - tick_price) < tick_size/2:
                        hist[closest_idx] += vol_per_tick
        
        # Sıfır hacimli seviyeleri filtrele
        non_zero_mask = hist > 0
        if not np.any(non_zero_mask):
            continue
        
        hist_nonzero = hist[non_zero_mask]
        prices_nonzero = price_levels[non_zero_mask]
        
        # POC - En yüksek hacimli fiyat seviyesi
        poc_idx = np.argmax(hist_nonzero)
        poc_value = prices_nonzero[poc_idx]
        
        # Value Area hesaplama - TradingView algoritması
        total_volume = hist_nonzero.sum()
        target_volume = total_volume * (value_area_pct / 100)
        
        # Fiyata göre sırala
        sort_idx = np.argsort(prices_nonzero)
        prices_sorted = prices_nonzero[sort_idx]
        hist_sorted = hist_nonzero[sort_idx]
        
        # POC pozisyonu
        poc_pos = np.where(np.abs(prices_sorted - poc_value) < tick_size/2)[0]
        if len(poc_pos) == 0:
            poc_pos = np.argmin(np.abs(prices_sorted - poc_value))
        else:
            poc_pos = poc_pos[0]
        
        # Value Area genişletme - TradingView gibi
        value_area_volume = hist_sorted[poc_pos]
        value_area_mask = np.zeros(len(prices_sorted), dtype=bool)
        value_area_mask[poc_pos] = True
        
        lower_idx = poc_pos - 1
        upper_idx = poc_pos + 1
        
        while value_area_volume < target_volume:
            lower_vol = hist_sorted[lower_idx] if lower_idx >= 0 else -1
            upper_vol = hist_sorted[upper_idx] if upper_idx < len(hist_sorted) else -1
            
            # Hangi tarafta hacim daha yüksek?
            if lower_vol >= upper_vol and lower_idx >= 0:
                value_area_volume += lower_vol
                value_area_mask[lower_idx] = True
                lower_idx -= 1
            elif upper_idx < len(hist_sorted):
                value_area_volume += upper_vol
                value_area_mask[upper_idx] = True
                upper_idx += 1
            else:
                break
        
        # VAL ve VAH
        val = prices_sorted[value_area_mask].min()
        vah = prices_sorted[value_area_mask].max()
        
        # TradingView yuvarlama - 2 desimal
        poc = round(poc_value, 2)
        val = round(val, 2)
        vah = round(vah, 2)
        
        # Debug için
        print(f"Date: {start_date.strftime('%Y-%m-%d')}, POC: {poc}, VAL: {val}, VAH: {vah}, Total Vol: {total_volume:.0f}")
        
        # Sonuçları df_result'e ekle
        df_result.loc[idx, 'FRVP_POC'] = poc
        df_result.loc[idx, 'FRVP_VAL'] = val
        df_result.loc[idx, 'FRVP_VAH'] = vah
    
    return df_result

# Kullanım
df_result = calculate_tv_frvp_gpt(TICKER, df_raw, df_res, value_area_pct=68)
df_result

Ticker: ULKER.IS, Tick Size: 0.01, Sample Price: 77.6482925415039
Date: 2026-02-10, POC: 132.2, VAL: 130.8, VAH: 134.74, Total Vol: 11036130
Date: 2026-02-04, POC: 133.0, VAL: 132.5, VAH: 136.75, Total Vol: 32131111
Date: 2026-02-04, POC: 133.0, VAL: 132.5, VAH: 136.75, Total Vol: 32131111
Date: 2026-02-04, POC: 133.0, VAL: 132.5, VAH: 136.75, Total Vol: 32131111
Date: 2026-02-04, POC: 133.0, VAL: 132.5, VAH: 136.75, Total Vol: 32131111
Date: 2026-02-04, POC: 133.0, VAL: 132.5, VAH: 136.75, Total Vol: 32131111
Date: 2024-07-16, POC: 109.39, VAL: 100.51, VAH: 115.83, Total Vol: 2699571950


,TICKER,INTERVAL,RANGE_TYPE,RANGE_STARTDATE,RANGE_SIZE,HIGHEST_VALUE,HIGHEST_DATE,ROW_SIZE_A_H,CUTT_OFF_DATE,END_DATETIME,FRVP_POC,FRVP_VAL,FRVP_VAH
0,ULKER.IS,daily,1day,2026-02-10 00:00:00+03:00,2,138.00,2026-02-10 00:00:00+03:00,2,None,2026-02-11 00:00:00+03:00,132.20,130.80,134.74
1,ULKER.IS,daily,1week,2026-02-04 00:00:00+03:00,6,139.50,2026-02-04 00:00:00+03:00,6,None,2026-02-11 00:00:00+03:00,133.00,132.50,136.75
2,ULKER.IS,daily,1months,2026-01-11 00:00:00+03:00,23,139.50,2026-02-04 00:00:00+03:00,6,None,2026-02-11 00:00:00+03:00,133.00,132.50,136.75
3,ULKER.IS,daily,3months,2025-11-11 00:00:00+03:00,66,139.50,2026-02-04 00:00:00+03:00,6,None,2026-02-11 00:00:00+03:00,133.00,132.50,136.75
4,ULKER.IS,daily,6months,2025-08-11 00:00:00+03:00,131,139.50,2026-02-04 00:00:00+03:00,6,None,2026-02-11 00:00:00+03:00,133.00,132.50,136.75
5,ULKER.IS,daily,1year,2025-02-11 00:00:00+03:00,252,139.50,2026-02-04 00:00:00+03:00,6,None,2026-02-11 00:00:00+03:00,133.00,132.50,136.75
6,ULKER.IS,daily,2year,2024-02-11 00:00:00+03:00,500,174.64,2024-07-16 00:00:00+03:00,399,None,2026-02-11 00:00:00+03:00,109.39,100.51,115.83
